In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.5"
export JULIA_PKG_PRECOMPILE_AUTO=0
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  echo "Installing PyCall.jl..."
  julia -e 'using Pkg; Pkg.add("PyCall"); Pkg.build("PyCall")'
  julia -e 'println("Success")'

fi

Installing Julia 1.8.5 on the current Colab Runtime...
2023-03-28 15:56:42 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.5-linux-x86_64.tar.gz [130873886/130873886] -> "/tmp/julia.tar.gz" [1]
Installing PyCall.jl...
  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed VersionParsing ── v1.3.0
   Installed Conda ─────────── v1.8.0
   Installed Parsers ───────── v2.5.8
   Installed SnoopPrecompile ─ v1.0.3
   Installed PyCall ────────── v1.95.1
   Installed Preferences ───── v1.3.0
   Installed MacroTools ────── v0.5.10
   Installed JSON ──────────── v0.21.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [438e738f] + PyCall v1.95.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8f4d0f93] + Conda v1.8.0
  [682c06a0] + JSON v0.21.3
  [1914dd2f] + MacroTools v0.5.10
  [69de0a69] + Parsers v2.5.8
  [21216c6a] + Preferences v1.3.0
  [438e738f] + 

Install PySR and PyTorch-Lightning:

In [2]:
%pip install -Uq pysr pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.8 MB/s eta 0:00:00


The following step is not normally required, but colab's printing is non-standard and we need to manually set it up PyJulia:


In [3]:
from julia import Julia

julia = Julia(compiled_modules=False, threads='auto')
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()

Let's install the backend of PySR, and all required libraries.

**(This may take some time)**

In [4]:
import pysr

# We don't precompile in colab because compiled modules are incompatible static Python libraries:
pysr.install(precompile=False)

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      Ubuntu 20.04.5 LTS
  uname: Linux 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz: 
              speed         user         nice          sys         idle          irq
       #1  2200 MHz        903 s          0 s        267 s       2124 s          0 s
       #2  2200 MHz        920 s          0 s        267 s       2023 s          0 s
  Memory: 12.681198120117188 GB (11673.60546875 MB free)
  Uptime: 337.44 sec
  Load Avg:  1.64  0.83  0.36
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 1 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_PROJECT = @pysr-0.12.1
  JULIA_PKG_PRECOMPILE_AUTO = 0
  TCLLIBPATH = /usr/share/tcltk/tcllib1.20
  HOME = /root
  PYTHONPATH = /env/python
  LIBRARY_PATH = /usr/local/cuda/lib64/stubs
  PATH

[ Info: Julia version info
[ Info: Julia executable: /usr/local/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /usr/bin/python3
│     libpython: /usr/lib/x86_64-linux-gnu/libpython3.9.so.1.0
│ Python:
│     python: /usr/bin/python3
└     libpython: 
     Cloning git-repo `https://github.com/MilesCranmer/SymbolicRegression.jl`
     Cloning git-repo `https://github.com/JuliaParallel/ClusterManagers.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed SIMDTypes ──────────────────────── v0.1.0
   Installed GPUArraysCore ──────────────────── v0.1.4
   Installed DynamicPolynomials ─────────────── v0.4.6
   Installed RealDot ────────────────────────── v0.1.0
   Installed BitTwiddlingConvenienceFunctions ─ v0.1.5
   Installed MutableArithmetics ─────────────── v1.2.3
   Installed IRTools ────────────────────────── v0.4.9
   Installed Dynam

In [5]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split

In [6]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Python_assignmentSpecial.csv to Python_assignmentSpecial.csv


In [7]:
import pandas as pd
import io
import math
df = pd.read_csv(io.BytesIO(uploaded['Python_assignmentSpecial.csv']))
print(df)

            a             b           c             d
0    1.575620  4.000000e+00   54.841471  6.123234e-17
1    0.684956  6.400000e+01   66.605745 -1.000000e+00
2    0.869612  3.240000e+02   74.042351 -1.836970e-16
3    1.639972  1.024000e+03   79.676245  1.000000e+00
4    0.761846  2.500000e+03   84.989993  3.061617e-16
..        ...           ...         ...           ...
195  1.379550  5.903156e+09  271.334888  1.000000e+00
196  0.617608  6.024554e+09  271.634960  3.870812e-14
197  1.082557  6.147814e+09  271.201872 -1.000000e+00
198  1.515702  6.272957e+09  270.840462  7.354690e-15
199  0.653469  6.400000e+09  271.288299  1.000000e+00

[200 rows x 4 columns]


In [8]:
df_a=pd.DataFrame(df['a'])
df_a

,a
0,1.575620
1,0.684956
2,0.869612
3,1.639972
4,0.761846
...,...
195,1.379550
196,0.617608
197,1.082557
198,1.515702


In [9]:
df_b=pd.DataFrame(df['b'])
df_c=pd.DataFrame(df['c'])
df_d=pd.DataFrame(df['d'])

In [10]:
myRange = np.arange(0,200,1)
df_n = pd.DataFrame({"no": myRange})

In [11]:
model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*", "-", "/","^"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        "log",
        "tan",
        "sqrt",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

Model for Column A

In [12]:
model.fit(df_n, df_a)

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1289: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:220: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.260e+04
Head worker occupation: 24.7%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 35 / 600 total iterations (5.833%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.357e-01  1.097e-07  1.0639428
4           2.691e-02  5.393e-01  tan(cos(sin(no)))
5           2.643e-02  1.791e-02  inv(tan(cos(cos(no))))
6           3.744e-03  1.954e+00  tan(cos(sin(no + 0.22109656)))
7           1.938e-03  6.588e-01  inv(tan(cos(cos(no + 0.19125427))))
8           1.879e-03  3.054e-02  inv(tan(cos(cos(no + tan(0.19125427)))))
9           1.630e-03  1.425e-01  inv(tan(cos(cos((no + -0.03819972) + 0.25340325))))
10          1.630e-03  4.649e-06  inv(tan(cos(cos((no + 0.25340325) + tan(-0.03819972)))))
11          3.000e-04  1.692e+00  (tan(cos(sin(no - -0.2146286))) * in

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          1.0639428   
	1         0.539286                                  tan(cos(sin(no)))   
	2         0.017907                             inv(tan(cos(cos(no))))   
	3         1.961150                     tan(cos(sin(no + 0.21407211)))   
	4         0.816970                inv(tan(cos(cos(no + 0.21990609))))   
	5         2.431770        tan(cos(sin(no - -0.21995169)) * 1.0299886)   
	6         0.063314   tan(cos(sin(no - sin(-0.21995169))) * 1.0299886)   
	7         0.039506  tan(cos(sin(no - sin(sin(-0.21995169)))) * 1.0...   
	8         0.081475  tan((cos(sin(no - sin(-0.21995169))) * 1.02998...   
	9         0.025966  (tan(sqrt(cos(sin(no - sin(-0.21995169)))) ^ e...   
	10        0.327869  (tan(sqrt(cos(sin(no - -0.21459359))) ^ exp(ta...   
	11        0.003411  (tan(sqrt(cos(sin(no - -0.21459359))) ^ exp(sq...   
	12        0.215341  (tan(sqrt(cos(sin(no - -0.23135813))) ^ exp(co...   
	13        0.423827  (tan(sqrt(cos(sin(no - -0.21995169))) ^ exp(co...   
	14  >>>>  1.027134  (tan(sqrt(cos(sin(no - -0.21995169))) ^ exp(co...   
	15        0.181560  (tan(sqrt(cos(sin(no - -0.21995169))) ^ exp(co...   
	
	        loss  complexity  
	0   0.135693           1  
	1   0.026911           4  
	2   0.026433           5  
	3   0.003719           6  
	4   0.001643           7  
	5   0.000144           8  
	6   0.000136           9  
	7   0.000130          10  
	8   0.000120          11  
	9   0.000114          13  
	10  0.000082          14  
	11  0.000082          15  
	12  0.000066          16  
	13  0.000043          17  
	14  0.000015          18  
	15  0.000011          20  
]

In [13]:
print(model)

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          1.0639428   
	1         0.539286                                  tan(cos(sin(no)))   
	2         0.017907                             inv(tan(cos(cos(no))))   
	3         1.961150                     tan(cos(sin(no + 0.21407211)))   
	4         0.816970                inv(tan(cos(cos(no + 0.21990609))))   
	5         2.431770        tan(cos(sin(no - -0.21995169)) * 1.0299886)   
	6         0.063314   tan(cos(sin(no - sin(-0.21995169))) * 1.0299886)   
	7         0.039506  tan(cos(sin(no - sin(sin(-0.21995169)))) * 1.0...   
	8         0.081475  tan((cos(sin(no - sin(-0.21995169))) * 1.02998...   
	9         0.025966  (tan(sqrt(cos(sin(no - sin(-0.21995169)))) ^ e...   
	10        0.327869  (tan(sqrt(cos(sin(no - -0.21459359))) ^ exp(ta...   
	11        0.003411  (tan(sqrt(cos(sin(no - -0.21459359))) ^ exp(sq...   
	12      

In [14]:
t=[1]*200
def model_a():
    for i in range(0,200):
        t[i]=math.tan(math.cos(math.sin(int(i) + 0.21628661)) * 1.0291212)
model_a()
df_a['model_a']=t
df_a

,a,model_a
0,1.575620,1.576460
1,0.684956,0.697099
2,0.869612,0.873319
3,1.639972,1.651209
4,0.761846,0.769948
...,...,...
195,1.379550,1.363827
196,0.617608,0.632104
197,1.082557,1.073733
198,1.515702,1.509365


Model for Column B

In [15]:
model.fit(df_n, df_b)

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1762: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1289: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:220: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 3.380e+04
Head worker occupation: 2.0%
Progress: 94 / 600 total iterations (15.667%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           4.654e+18  -4.477e-01  -1.2122245
3           4.650e+18  3.986e-04  exp(tan(1.5003628))
5           4.620e+18  3.312e-03  exp(tan(no - -2.0173))
6           4.277e+18  7.708e-02  exp(exp(exp(exp(exp(-2.5968704)))))
7           6.948e+16  4.120e+00  (no * (no * (no * 714.3597)))
9           3.740e+13  3.764e+00  ((no * (no * (no * no))) / 0.24443981)
11          7.170e+11  1.977e+00  ((no * (no * (no * (no - -4.525196)))) / 0.25073996)
15          2.594e+09  1.405e+00  (((no - -3.52492) * (no + no)) * ((no * (no + no)) + no))
20          4.270e+08  3.608e-01  ((no * ((((no * 2.9989586) + no) - (sqrt(cos(cos(-0.56017727))...
                                  ) ^ no)) + 16.322649)) * (no * no))
-----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                           116396.4   
	1        3.439782                                    (no ^ 4.271322)   
	2        1.406360                     ((no + 11.954531) ^ 4.2180324)   
	3        1.202813                ((no + exp(2.3832843)) ^ 4.2232494)   
	4        0.504620                     (((no + no) * no) ^ 2.0019686)   
	5        0.165969             ((no * (no * (no * no))) / 0.24443981)   
	6        1.977087  ((no * (no * (no * (no - -4.525196)))) / 0.250...   
	7        1.904907  ((no * (no * ((no - inv(-0.23975629)) * no))) ...   
	8  >>>>  5.521852  ((no * (((no * 2.9989586) + no) + 16.322649)) ...   
	
	           loss  complexity  
	0  4.653892e+18           1  
	1  4.786960e+15           3  
	2  2.874162e+14           5  
	3  8.632496e+13           6  
	4  5.211738e+13           7  
	5  3.739590e+13           9  
	6  7.170475e+11          11  
	7  1.067228e+11          12  
	8  4.267245e+08          13  
]

In [16]:
print(model)

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                           116396.4   
	1        3.439782                                    (no ^ 4.271322)   
	2        1.406360                     ((no + 11.954531) ^ 4.2180324)   
	3        1.202813                ((no + exp(2.3832843)) ^ 4.2232494)   
	4        0.504620                     (((no + no) * no) ^ 2.0019686)   
	5        0.165969             ((no * (no * (no * no))) / 0.24443981)   
	6        1.977087  ((no * (no * (no * (no - -4.525196)))) / 0.250...   
	7        1.904907  ((no * (no * ((no - inv(-0.23975629)) * no))) ...   
	8  >>>>  5.521852  ((no * (((no * 2.9989586) + no) + 16.322649)) ...   
	
	           loss  complexity  
	0  4.653892e+18           1  
	1  4.786960e+15           3  
	2  2.874162e+14           5  
	3  8.632496e+13           6  
	4  5.211738e+13           7  
	5  3.739590e+13           9  
	6  7.170475e+11      

In [17]:
x=[]
def model_b():
    for i in range(1,201):
        x.append(i*(((i+i)*i)*(i+i)))
    
model_b()
df_b['model_b']=x
df_b

,b,model_b
0,4.000000e+00,4
1,6.400000e+01,64
2,3.240000e+02,324
3,1.024000e+03,1024
4,2.500000e+03,2500
...,...,...
195,5.903156e+09,5903156224
196,6.024554e+09,6024553924
197,6.147814e+09,6147814464
198,6.272957e+09,6272956804


Model for Column C

In [22]:
model.fit(df_n, df_c)

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1762: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1289: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:220: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 4.180e+04
Head worker occupation: 2.2%
Progress: 116 / 1200 total iterations (9.667%)
Best equations for output 1
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           2.532e+03  -1.000e-10  206.96709
3           2.812e+02  1.099e+00  (no + 107.46712)
5           2.812e+02  2.086e-07  ((no + 100.077896) + 7.389688)
6           2.786e+02  9.046e-03  ((no * cos(-0.40121454)) - -107.46704)
7           2.593e+02  7.185e-02  (((no + 55.66798) + 55.66798) / 1.0356307)
8           2.535e+02  2.275e-02  (((no * 0.95045394) + sin(0.7937703)) - -107.46704)
9           1.932e+02  2.714e-01  ((no * ((-19.799397 / 99.34797) - -1.0368688)) + 123.62831)
11          1.853e+02  2.090e-02  ((((inv(0.010714002) - -68.68376) + no) + sqrt(no)) / 1.294677...
                                  9)
15          1.800e+02  7.351e-03  ((((((no + 106.3504) + tan(exp(0.398719

PySRRegressor.equations_ = [
[
	    pick         score                                           equation  \
	0         0.000000e+00                                          206.96709   
	1         1.098816e+00                                   (no + 107.46712)   
	2         1.969780e-01                    ((no - -142.79079) ^ 0.9714611)   
	3         2.635739e+00                ((sqrt(no) * 14.930464) + 66.74445)   
	4         1.103801e-06  ((72.31778 + (sqrt(no) * 14.930567)) + -5.574422)   
	5         2.508698e-02    ((66.70355 + (sqrt(no) * 14.935107)) + cos(no))   
	6         1.587528e+00  (((sqrt(no) ^ 0.79614705) + 1.4412041) * (25.3...   
	7         5.052432e-01  (((sqrt(no) * 19.920158) - (-71.99634 - -22.57...   
	8         1.013907e-07  ((((13.233954 - -13.19844) - (no * 0.29690513)...   
	9         1.724070e-01  (((exp(cos(no)) - (-23.613375 - 24.812353)) - ...   
	10        1.724331e-01  ((((cos(no) - (-23.492643 - 24.69159)) - (no *...   
	11        3.353166e-03  ((((sin(cos(no)) - (-23.492643 - 24.69159)) - ...   
	12  >>>>  2.480956e-01  ((((sin(no - -1.0922502) - (-23.492643 - 24.69...   
	
	           loss  complexity  
	0   2531.629600           1  
	1    281.177760           3  
	2    189.621720           5  
	3     13.589422           6  
	4     13.589392           8  
	5     13.252716           9  
	6      2.709258          10  
	7      0.986283          12  
	8      0.986283          14  
	9      0.698636          16  
	10     0.587982          17  
	11     0.586014          18  
	12     0.457258          19  
], [
	   pick     score                                           equation  \
	0        0.000000                                          206.90355   
	1        1.303669                                  (no - -107.40291)   
	2        0.247051                   ((no * 0.85215646) - -122.11409)   
	3        4.736188                ((sqrt(no) * 14.97626) - -66.25085)   
	4        0.015366  ((sqrt(no) * 14.97626) - (-66.25085 + (0.73157...   
	5  >>>>  3.396008  ((sqrt(no - sin(no / 1.2347175)) * 14.97626) -...   
	6        0.010062  ((sqrt((no - sin(no / 1.2347175)) - -8.865871e...   
	
	          loss  complexity  
	0  2534.517600           1  
	1   186.871500           3  
	2   114.013720           5  
	3     1.000131           6  
	4     0.940508          10  
	5     0.031513          11  
	6     0.030886          13  
]]

In [23]:
print(model)

PySRRegressor.equations_ = [
[
	    pick         score                                           equation  \
	0         0.000000e+00                                          206.96709   
	1         1.098816e+00                                   (no + 107.46712)   
	2         1.969780e-01                    ((no - -142.79079) ^ 0.9714611)   
	3         2.635739e+00                ((sqrt(no) * 14.930464) + 66.74445)   
	4         1.103801e-06  ((72.31778 + (sqrt(no) * 14.930567)) + -5.574422)   
	5         2.508698e-02    ((66.70355 + (sqrt(no) * 14.935107)) + cos(no))   
	6         1.587528e+00  (((sqrt(no) ^ 0.79614705) + 1.4412041) * (25.3...   
	7         5.052432e-01  (((sqrt(no) * 19.920158) - (-71.99634 - -22.57...   
	8         1.013907e-07  ((((13.233954 - -13.19844) - (no * 0.29690513)...   
	9         1.724070e-01  (((exp(cos(no)) - (-23.613375 - 24.812353)) - ...   
	10        1.724331e-01  ((((cos(no) - (-23.492643 - 24.69159)) - (no *...   
	11        3.353166e-03  ((((sin(

In [32]:
c=[]
def model_c():
    for no in range(1,201):
        c.append((((math.sin(no - -1.0922502) - (-23.492643 - 24.69159)) - (no * 0.29687312)) + (math.sqrt(no) * 19.919636)) + 1.2512596)
model_c()
df_c['model_c']=c
df_c

,c,model_c
0,54.841471,69.925351
1,66.605745,77.061688
2,74.042351,82.232897
3,79.676245,87.158556
4,84.989993,92.303010
...,...,...
195,271.334888,270.859904
196,271.634960,270.365835
197,271.201872,270.027243
198,270.840462,270.534083


Model for Column D

In [33]:
model.fit(df_n, df_d)

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1762: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1289: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:220: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 3.400e+04
Head worker occupation: 1.9%
Progress: 94 / 1200 total iterations (7.833%)
Best equations for output 1
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           5.000e-01  1.787e-07  9.217298e-6
4           5.000e-01  1.327e-05  (-0.0030554878 * sin(no))
5           4.979e-01  4.218e-03  (-0.054021634 / (-0.7997943 + no))
6           4.922e-01  1.148e-02  ((sin(no) ^ no) * -0.62085396)
8           4.909e-01  1.273e-03  (((sin(no) ^ no) ^ no) * -0.7335158)
14          4.888e-01  7.191e-04  tan(cos((1.1091295 ^ sqrt(no)) * no) / ((-0.02339832 / 1.55102...
                                  23) - no))
17          4.886e-01  1.550e-04  tan((cos(sin(1.1091295)) / ((1.1091295 - ((cos(no) * no) * no)...
                                  ) / -0.52194583)) * no)
18          3.903e-01  2.247e-01  (log(cos(sin(log(tan(cos(sin((0.0073840534 - (no * 0.3

PySRRegressor.equations_ = [
[
	   pick      score                                           equation  \
	0         0.000000                                        9.217298e-6   
	1         0.000019                                (3.7783284e-5 * no)   
	2         0.020438                            (0.002571017 / cos(no))   
	3         0.028035                         sin(0.012549523 / cos(no))   
	4  >>>>  23.728191                       (sin(no * 1.5707964) * -1.0)   
	5         0.012787  sin(((no - 1.5206765) * 1.5707964) - cos(cos(c...   
	
	           loss  complexity  
	0  4.999999e-01           1  
	1  4.999810e-01           3  
	2  4.898662e-01           4  
	3  4.763235e-01           5  
	4  2.359824e-11           6  
	5  2.076564e-11          16  
], [
	   pick      score                                           equation  \
	0         0.000000                                       3.5039257e-6   
	1         0.000019                                 (3.778278e-5 * no)   
	2  >>>>  23.743115                               sin(no * -1.5707954)   
	3         0.013664  sin(((sin(0.45374593) ^ sqrt(0.37757775)) * ta...   
	
	           loss  complexity  
	0  4.999999e-01           1  
	1  4.999811e-01           3  
	2  2.440336e-11           4  
	3  2.015456e-11          18  
]]

In [34]:
print(model)

PySRRegressor.equations_ = [
[
	   pick      score                                           equation  \
	0         0.000000                                        9.217298e-6   
	1         0.000019                                (3.7783284e-5 * no)   
	2         0.020438                            (0.002571017 / cos(no))   
	3         0.028035                         sin(0.012549523 / cos(no))   
	4  >>>>  23.728191                       (sin(no * 1.5707964) * -1.0)   
	5         0.012787  sin(((no - 1.5206765) * 1.5707964) - cos(cos(c...   
	
	           loss  complexity  
	0  4.999999e-01           1  
	1  4.999810e-01           3  
	2  4.898662e-01           4  
	3  4.763235e-01           5  
	4  2.359824e-11           6  
	5  2.076564e-11          16  
], [
	   pick      score                                           equation  \
	0         0.000000                                       3.5039257e-6   
	1         0.000019                                 (3.778278e-5 * no)   
	2  >

In [37]:
d=[]
def model_d():
    for no in range(0,200):
        d.append(math.sin(((no - 1.5206765) * 1.5707964) - math.cos(math.cos(math.cos(math.cos(-0.9384193 + (0.69160765 / -0.49479094)))))))
model_d()
df_d['model_d']=d
df_d

,d,model_d
0,6.123234e-17,0.000008
1,-1.000000e+00,-1.000000
2,-1.836970e-16,-0.000008
3,1.000000e+00,1.000000
4,3.061617e-16,0.000007
...,...,...
195,1.000000e+00,1.000000
196,3.870812e-14,-0.000007
197,-1.000000e+00,-1.000000
198,7.354690e-15,0.000007
